In [5]:
import os
import numpy as np
from time import time
import datetime
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from math import sqrt
import pandas as pd

filepath ='/home/dell/Xinda/SVM/server/Audio/data_vggish/vggish_features_a_deleteNull.csv'
data = pd.read_csv(filepath)
x = data.iloc[:, :-1]  # 数据特征
y = data.iloc[:,-1]  # 标签

In [6]:

# 将数据划分为训练集和测试集，test_size=.3表示30%的测试集, 随机数种子, 保证可复现性
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=423)

# 修正测试集和训练集的索引
for i in [x_train, x_test, y_train, y_test ]:
    i.index  = range(i.shape[0])

# 标准化
scaler_x = StandardScaler()
x_train_std = scaler_x.fit_transform(x_train)
x_test_std = scaler_x.fit_transform(x_test)
print(x_train_std.shape)
print(y_train.shape)



def getPvar(vals, mean):
    N = len(vals)
    su = 0
    for i in range(len(vals)):
        su = su + ((vals[i]-mean)*(vals[i]-mean))
    pvar = (1/N) * su
    return pvar

def getMean(vals):
    su = 0
    for i in range(len(vals)):
        su = su + vals[i]
    mean = su/(len(vals))
    return mean

def getMeanofDiffs(xvals, yvals):
    su = 0
    for i in range(len(xvals)):
        su = su + ((xvals[i] - yvals[i])*(xvals[i] - yvals[i]))
    meanodiffs = su/(len(xvals))
    return meanodiffs

def getCCC(pvarfe,pvarexp,meanofdiff,meanfe,meanexp):
    bottom = pvarfe + pvarexp + ((meanfe - meanexp)*(meanfe - meanexp))
    answer = 1 - (meanofdiff / bottom)
    return answer


(10028, 384)
(10028,)


## Gamma-1

In [ ]:
times_all = time()
# 调试两个参数 gamma & C  ，默认情况下C为1，通常来说这都是一个合理的参数。
RMSE_list = []
Spearman_list = []
CCC_list = []

# =========================================== Gamma
gamma_range = np.logspace(-10, 1, 10, base=2)  # 返回13个数字，底是2
print("gamma_rang:", gamma_range)

for gamma_item in gamma_range:
    count = 1
    time0 = time()
    print("Start-{0}, gamma={1}".format(count, gamma_item))
    count = count + 1
    clf = SVR(kernel='rbf', gamma=gamma_item, cache_size=5000)
    clf.fit(x_train_std, y_train)

    result = clf.predict(x_test_std)
    # accuracy
    rmse = sqrt(mean_squared_error(y_test, result))
    RMSE_list.append(rmse)
    print("(1) Evaluation - RMSE = ", rmse)

    # Spearman
    data = {'result': result, 'y_test': y_test}
    df = pd.DataFrame(data, columns=['result', 'y_test'])
    spearman = df.corr(method="spearman")
    print("(2) Evaluation - Spearmman = \n", spearman)
    Spearman_list.append(spearman)

    # CCC
    prediction = result
    ground = y_test
    meanfe = getMean(ground)
    meanexp = getMean(prediction)
    meanofdiff = getMeanofDiffs(ground, prediction)
    pvarfe = getPvar(ground, meanfe)
    pvarexp = getPvar(prediction, meanexp)
    ccc = getCCC(pvarfe, pvarexp, meanofdiff, meanfe, meanexp)
    CCC_list.append(ccc)
    print('(3) Evaluation - CCC =  ' + str(ccc))
    print(datetime.datetime.fromtimestamp(time() - time0).strftime("%M:%S:%f"))
    print()
    print()

print("Gamma = ", gamma_range[RMSE_list.index(min(RMSE_list))])
print("RMSE = ", min(RMSE_list))
print("Spearman = ", Spearman_list[RMSE_list.index(min(RMSE_list))])
print("CCC = ", CCC_list[RMSE_list.index(min(RMSE_list))])
print(datetime.datetime.fromtimestamp(time() - times_all).strftime("%M:%S:%f"))

In [ ]:
times_all = time()
# 调试两个参数 gamma & C  ，默认情况下C为1，通常来说这都是一个合理的参数。
RMSE_list = []
Spearman_list = []
CCC_list = []

# =========================================== Gamma
C_range = np.logspace(-10, 1, 10, base=2)  # 返回13个数字，底是2
print("gamma_rang:", gamma_range)

for C_item in C_range:
    count = 1
    time0 = time()
    print("Start-{0}, c={1}".format(count, C_item))
    count = count + 1
    clf = SVR(kernel='rbf', gamma=0.005315585938181161,C=C_item , cache_size=5000)
    clf.fit(x_train_std, y_train)

    result = clf.predict(x_test_std)
    # accuracy
    rmse = sqrt(mean_squared_error(y_test, result))
    RMSE_list.append(rmse)
    print("(1) Evaluation - RMSE = ", rmse)

    # Spearman
    data = {'result': result, 'y_test': y_test}
    df = pd.DataFrame(data, columns=['result', 'y_test'])
    spearman = df.corr(method="spearman")
    print("(2) Evaluation - Spearmman = \n", spearman)
    Spearman_list.append(spearman)

    # CCC
    prediction = result
    ground = y_test
    meanfe = getMean(ground)
    meanexp = getMean(prediction)
    meanofdiff = getMeanofDiffs(ground, prediction)
    pvarfe = getPvar(ground, meanfe)
    pvarexp = getPvar(prediction, meanexp)
    ccc = getCCC(pvarfe, pvarexp, meanofdiff, meanfe, meanexp)
    CCC_list.append(ccc)
    print('(3) Evaluation - CCC =  ' + str(ccc))
    print(datetime.datetime.fromtimestamp(time() - time0).strftime("%M:%S:%f"))
    print()
    print()

print("C = ", max(CCC_list))
print("RMSE = ",RMSE_list[CCC_list.index(max(CCC_list))])
print("Spearman = ", Spearman_list[CCC_list.index(max(CCC_list))])
print("CCC = ", max(CCC_list))
print(datetime.datetime.fromtimestamp(time() - times_all).strftime("%M:%S:%f"))

#%%

## prediction

In [7]:
RMSE_list = []
Spearman_list = []
CCC_list = []
clf = SVR(kernel = 'rbf', gamma = 0.005315585938181161, C=2, cache_size=5000)
clf.fit(x_train_std, y_train)
result = clf.predict(x_test_std)
# accuracy
rmse = sqrt(mean_squared_error(y_test, result))
print("(1) Evaluation - RMSE = ", rmse)

# Spearman
data = {'result':result, 'y_test':y_test}
df = pd.DataFrame(data, columns=['result','y_test'])
spearman = df.corr(method="spearman" )
print("(2) Evaluation - Spearmman = \n", spearman)

# CCC
prediction = result
ground = y_test
meanfe = getMean(ground)
meanexp = getMean(prediction)
meanofdiff = getMeanofDiffs(ground,prediction)
pvarfe = getPvar(ground, meanfe)
pvarexp = getPvar(prediction, meanexp)
ccc = getCCC(pvarfe,pvarexp,meanofdiff,meanfe,meanexp)
print('(3) Evaluation - CCC =  ' + str(ccc))

df = pd.DataFrame(data={"vggish_prediction_a": prediction, "vggish_groundtruth_a": y_test.values.tolist()})
df.to_csv("eval_vggish_arousal.csv")
print("save success!")

(1) Evaluation - RMSE =  0.3562565596720691
(2) Evaluation - Spearmman = 
           result    y_test
result  1.000000  0.255014
y_test  0.255014  1.000000
(3) Evaluation - CCC =  0.242967131003328
save success!
